In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

train_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/train.csv')
test_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/test.csv')

X = train_data[['Age', 'Gender', 'T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT', 'TP', 'Alb', 'AG_ratio']]
y = train_data['disease']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_val)[:, 1]

auc_score = roc_auc_score(y_val, y_pred_prob)
print(f'AUC: {auc_score}')


ValueError: could not convert string to float: 'Male'

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# データ読み込み
train_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/train.csv')
test_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/test.csv')

# One-Hot Encoding for categorical variables like 'Gender'
train_data = pd.get_dummies(train_data, columns=['Gender'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Gender'], drop_first=True)

# 特徴量とターゲット変数の設定
X = train_data[['Age', 'T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT', 'TP', 'Alb', 'AG_ratio', 'Gender_Male']]
y = train_data['disease']

# データを訓練セットとテストセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ロジスティック回帰モデルの構築
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# テストデータで予測
y_pred_prob = model.predict_proba(X_val)[:, 1]

# AUCスコアの計算
auc_score = roc_auc_score(y_val, y_pred_prob)
print(f'AUC: {auc_score}')

# テストデータの予測
test_features = test_data[['Age', 'T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT', 'TP', 'Alb', 'AG_ratio', 'Gender_Male']]
test_pred_prob = model.predict_proba(test_features)[:, 1]

# 閾値0.5を用いて予測結果を0か1に変換
threshold = 0.5
test_pred = (test_pred_prob >= threshold).astype(int)

# 連番IDの生成
test_data['id'] = range(0, len(test_data))

# 予測結果を保存
output = pd.DataFrame({
    'id': test_data['id'],    # 生成した連番ID
    'prediction': test_pred  # 閾値に基づいた二値分類の予測結果
})

# CSVファイルとして保存（ヘッダーなし）
output.to_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/predictions.csv', index=False, header=False)
print("予測結果が保存されました。")


AUC: 0.9367406786761625
予測結果が保存されました。


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve

# データ読み込み
train_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/train.csv')
test_data = pd.read_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/test.csv')

# One-Hot Encoding for categorical variables like 'Gender'
train_data = pd.get_dummies(train_data, columns=['Gender'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Gender'], drop_first=True)

# 特徴量とターゲット変数の設定
X = train_data[['Age', 'T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT', 'TP', 'Alb', 'AG_ratio', 'Gender_Male']]
y = train_data['disease']

# データを訓練セットとテストセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ロジスティック回帰モデルの構築
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 検証データでの予測確率
y_pred_prob = model.predict_proba(X_val)[:, 1]

# AUCスコアの計算
auc_score = roc_auc_score(y_val, y_pred_prob)
print(f'AUC: {auc_score}')

# ROC曲線に基づく最適な閾値の算出
fpr, tpr, thresholds = roc_curve(y_val, y_pred_prob)

# 最適な閾値の選択: FPRとTPRの差が最も小さい点を探す
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]
print(f'最適な閾値: {optimal_threshold}')

# テストデータでの予測確率
test_features = test_data[['Age', 'T_Bil', 'D_Bil', 'ALP', 'ALT_GPT', 'AST_GOT', 'TP', 'Alb', 'AG_ratio', 'Gender_Male']]
test_pred_prob = model.predict_proba(test_features)[:, 1]

# 最適な閾値を用いて予測結果を0か1に変換
test_pred = (test_pred_prob >= optimal_threshold).astype(int)

# 連番IDの生成
test_data['id'] = range(0, len(test_data))

# 予測結果を保存
output = pd.DataFrame({
    'id': test_data['id'],    # 生成した連番ID
    'prediction': test_pred  # 閾値に基づいた二値分類の予測結果
})

# CSVファイルとして保存（ヘッダーなし）
output.to_csv('/home-asustor/teramoto/SIGNATE/Beginner/data/02predictions.csv', index=False, header=False)
print("予測結果が保存されました。")

# 検証データで精度の計算
y_val_pred = (y_pred_prob >= optimal_threshold).astype(int)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Accuracy: {accuracy}')


AUC: 0.9367406786761625
最適な閾値: 0.35839126632728985
予測結果が保存されました。
Accuracy: 0.8941176470588236
